In [1]:
print("hello")

hello


In [26]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyMuPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.llms import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI


In [ ]:
!pip install pypdf
!pip install pymupdf
!pip install openai
!pip install -U langchain-openai

In [5]:
def load_pdf(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyMuPDFLoader)

    documents=loader.load()

    return documents

In [6]:
extracted_data=load_pdf("data/")

In [ ]:
extracted_data

In [8]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)

    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)

In [10]:
len(text_chunks)

6975

In [14]:
import os
os.environ["OPENAI_API_KEY"]="sk-proj-EV1hw8qZlFkj8MUAogeXT3BlbkFJsAarOjFKBDuJ67kmx4Zy"
os.environ["PINECONE_API_KEY"]="81754309-acf1-4ac8-bba0-28f4da9d5426"

In [15]:
embedding=OpenAIEmbeddings()

In [16]:
embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000024B448F9090>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000024B44908590>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [17]:
len(text_chunks)

6975

In [18]:
print(text_chunks[0].page_content)

The GALE
ENCYCLOPEDIA
of MEDICINE
SECOND EDITION


In [19]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY','81754309-acf1-4ac8-bba0-28f4da9d5426')

In [20]:
PINECONE_API_KEY="81754309-acf1-4ac8-bba0-28f4da9d5426"
index_name='medical-chatbot'
index=pinecone.Index(api_key='81754309-acf1-4ac8-bba0-28f4da9d5426',host='https://medical-chatbot-x603g1g.svc.aped-4627-b74a.pinecone.io')

In [ ]:
docsearch=PineconeVectorStore.from_texts([t.page_content for t in text_chunks],embedding=embedding,index_name=index_name)

In [22]:
docsearch=Pinecone.from_existing_index(index_name,embedding)
query1="What are allergies?"
docs=docsearch.similarity_search(query1,k=3)
print("Result",docs)

Result [Document(page_content='Although environmental medicine is gaining more\nrespect within conventional medicine, detoxification\nKEY TERMS\nAllergen—A foreign substance, such as mites in\nhouse dust or animal dander, that when\ninhaled,causes the airways to narrow and pro-\nduces symptoms of asthma.\nAntibody—A protein, also called immunoglobu-\nlin, produced by immune system cells to remove\nantigens (the foreign substances that trigger the\nimmune response).\nFibromyalgia—A condition of debilitating pain,'), Document(page_content='repeated exposure to an allergen (an allergy-causing sub-\nstance) triggers an immune response that inflames the\nskin. Tens of thousands of drugs, pesticides, cosmetics,\nfood additives, commercial chemicals, and other sub-\nstances have been identified as potential allergens. Fewer\nthan 30, however, are responsible the majority of ACD\ncases. Common culprits include poison ivy, poison oak,\nand poison sumac; fragrances and preservatives in cosmet-')

In [23]:
docs

[Document(page_content='Although environmental medicine is gaining more\nrespect within conventional medicine, detoxification\nKEY TERMS\nAllergen—A foreign substance, such as mites in\nhouse dust or animal dander, that when\ninhaled,causes the airways to narrow and pro-\nduces symptoms of asthma.\nAntibody—A protein, also called immunoglobu-\nlin, produced by immune system cells to remove\nantigens (the foreign substances that trigger the\nimmune response).\nFibromyalgia—A condition of debilitating pain,'),
 Document(page_content='repeated exposure to an allergen (an allergy-causing sub-\nstance) triggers an immune response that inflames the\nskin. Tens of thousands of drugs, pesticides, cosmetics,\nfood additives, commercial chemicals, and other sub-\nstances have been identified as potential allergens. Fewer\nthan 30, however, are responsible the majority of ACD\ncases. Common culprits include poison ivy, poison oak,\nand poison sumac; fragrances and preservatives in cosmet-'),
 Doc

In [24]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer,just say that you don't know ,don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [25]:
PROMPT=PromptTemplate(input_variables=["context","question"],template=prompt_template)
chain_type_kwargs={"prompt":PROMPT}

In [27]:
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.5,
    openai_api_key="sk-proj-EV1hw8qZlFkj8MUAogeXT3BlbkFJsAarOjFKBDuJ67kmx4Zy"
)

In [28]:
qa=RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",retriever=docsearch.as_retriever(),return_source_documents=True,chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
	user_input1=input(f"Input Prompt:")
	result=qa({"query":user_input1})
	print("Response : ",result["result"])